## Setup and configuration

References: https://github.com/pmarcelino/blog/blob/master/dogs_cats/dogs_cats.ipynb

**Kaggle API setup**

In [0]:
!mkdir .kaggle

import json
token = {"username":"npqbuu","key":"224292a46181101043b9eaa22bc77a39"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!chmod 600 /root/.kaggle/kaggle.json

- path is now set to: {/content}


In [0]:
# Download dataset from Kaggle

!kaggle datasets download -d moltean/fruits -p /content
!unzip \*.zip

**Import datas**

In [0]:
# Base variables
base_dir = 'fruits-360/'
train_dir = os.path.join(base_dir, 'Training')
validation_dir = os.path.join(base_dir, 'Training')
test_dir = os.path.join(base_dir, 'Test')
Banana_dir = os.path.join(train_dir, 'Banana')
Avocado_dir = os.path.join(train_dir, 'Avocado')

train_size, validation_size, test_size = 200, 100, 100

img_width, img_height = 224, 224  # Default input size for VGG16

In [0]:
# Show pictures
import os, random
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing import image

def show_pictures(path):
    random_img = random.choice(os.listdir(path))
    img_path = os.path.join(path, random_img)

    img = image.load_img(img_path, target_size=(img_width, img_height))
    img_tensor = image.img_to_array(img)  # Image data encoded as integers in the 0–255 range
    img_tensor /= 255.  # Normalize to [0,1] for plt.imshow application
    plt.imshow(img_tensor)
    plt.show()
    
for i in range(0,2):
    show_pictures(Banana_dir)
    show_pictures(Avocado_dir)

### Extract features

In this example, we want to use a solution based on pre-trained models. We will use models that are composed of two parts:

- Convolutional base.
- Classifier.
Our approach will use the convolutional base to extract features, using them to train a classifier to classify the input image. Therefore, the features extracted from the convolutional base will be the same for all classifiers studied in this example.

Now let's see how to extract features from a convolutional base.

In [0]:
# Instantiate convolutional base
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet', 
                  include_top=False,
                  input_shape=(img_width, img_height, 3))  # 3 = number of channels in RGB pictures

In [0]:
# Check architecture
conv_base.summary()

In [0]:
# Extract features
import os, shutil
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
batch_size = 32

def extract_features(directory, sample_count, subset):
    features = np.zeros(shape=(sample_count, 7, 7, 512))  # Must be equal to the output of the convolutional base
    labels = np.zeros(shape=(sample_count))
    # Preprocess data
    generator = datagen.flow_from_directory(directory,
                                            target_size = (img_width,img_height),
                                            batch_size = batch_size,
                                            class_mode = 'categorical',
                                            color_mode = 'rgb',
                                            subset = subset)
    # Pass data through convolutional base
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size: (i + 1) * batch_size] = features_batch
        labels[i * batch_size: (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels

train_features, train_labels = extract_features(train_dir, train_size, 'training')  # Agree with our small dataset size
validation_features, validation_labels = extract_features(validation_dir, validation_size, 'validation')
test_features, test_labels = extract_features(test_dir, test_size, None)

Ok, now that we have the convolutional base, we need to pass our images through it for feature extraction.